<a href="https://colab.research.google.com/github/gt-cse-6040/bootcamp/blob/main/Module%201/Session%209/m1s9nb2_Data_Debugging_part2%20-%20Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Debugging Bad Solutions: Module 1, Part 2

From Midterm 2, Fall 2022 - Two and Three-Point Exercises

## Introduction

In [12]:
!pip install dill

!mkdir resources
%cd resources

!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/resources/tc_1
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/resources/tc_3
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/resources/tc_5

%cd ..

!mkdir tester_fw
%cd tester_fw

!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/tester_fw/__init__.py
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/tester_fw/test_utils.py
!wget https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/tester_fw/testers.py

%cd ..

/content/resources
--2023-12-24 17:51:48--  https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/resources/tc_1
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8159884 (7.8M) [text/plain]
Saving to: ‘tc_1’

tc_1                100%[===================>]   7.78M  --.-KB/s    in 0.07s   

2023-12-24 17:51:48 (113 MB/s) - ‘tc_1’ saved [8159884/8159884]

--2023-12-24 17:51:48--  https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/Module%201/Session%209/resources/tc_3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

### Purpose

On the exams you may initially write solutions that do not pass the test cases. That's okay! You will need to debug your code to determine what is causing the issue(s) and then figure out to how fix them. So how can we get better at debugging? We practice!

Below are exercises from the Fall 2022 Midterm 2. We have pre-written solutions for each exercise that are "bad" in one or more ways. Our solutions may contain one or more logic and/or syntax errors. . Can you find and fix the issues in each exercise and pass all of the test cases?

**Debugging your code:** Right before each exercise test cell, there is a block of text explaining the variables available to you for debugging. You may use these to test your code and can print/display them as needed (careful when printing large objects, you may want to print the head or chunks of rows at a time).

**Exercise point breakdown:**

- Exercise 1: **2** point(s)
- Exercise 3: **2** point(s)
- Exercise 5: **3** point(s)

## Scenario (don't dwell on this)

You have just been hired by the hot new startup **Spot-i-flix-ify** (this is a fictional company which will offer video and audio streaming services) as a Data Scientist. This is a small startup so you have to "wear many different hats," so to speak. Your first task on the job is to set up their data warehousing so that they can capture a historical record of their operations for analysis later. The operational database (which someone else has already set up) only contains the current state of the operation to maintain maximum efficiency while performing tasks like adding new customers, changing services, applying promotions, etc. It will not contain any history and is not intended have complex queries run against it.
  
While this is a fictional company and simulation data, **there is a real-world use case** for the processes developed in this notebook.

## On data types
These tables are made available to you in a staging environment as Pandas `DataFrame` objects. **All columns in all of the DataFrames are strings (even the columns where you would expect other data types)**.

## On SQL
We used Pandas exclusively in developing this problem, however some exercise are solvable using SQL. In the cell below we have included the function `dfs_to_conn` which can be used to create in-memory database connections. If you pass in a dictionary mapping table names to DataFrames, `dfs_to_conn` will return a sqlite 3 connection with all of the data in the DataFrames available under the names given as keys. You are also free to write to the in-memory database by creating tables, inserting/deleting/updating records, etc. Anything that SQLite allows should work!

Example:
  ```
my_df = pd.DataFrame({'A':[1,2,3], 'B': [4,5,6], 'C':['x', 'y', 'z']})
print(my_df)
#    A  B  C
# 0  1  4  x
# 1  2  5  y
# 2  3  6  z
conn = dfs_to_conn({'my_table': my_df})
cur = conn.cursor()
cur.execute('select A, B, C from my_table')
result = cur.fetchall()
conn.close()
print(result) # list of tuples, each tuple is a row
#[(1, 4, 'x'), (2, 5, 'y'), (3, 6, 'z')]

In [2]:
### Global Imports
###
### AUTOGRADER TEST - DO NOT REMOVE
###
import pandas as pd
import time
overall_start = time.time()

def dfs_to_conn(conn_dfs, index=False):
    import sqlite3
    conn = sqlite3.connect(':memory:')
    for table_name, df in conn_dfs.items():
        df.to_sql(table_name, conn, if_exists='replace', index=index)
    return conn

In [10]:
# !python -m pip install -r resources/requirements.txt

## Data (Don't dwell on this. The structures you are working with will be explained in each exercise.)
You are working with four tables:
- `customers` - Center of the "star" schema. Primary Key: `id`.
  - `customers.id` - a unique identifier for an individual customer.
  - `customers.paid` - ('True'|'False') - indicates whether a customer has paid their bill for their upcoming month of service.
- `prices` - The prices of the services offered by **Spot-i-flix-ify**. Primary Key: `service, tier, promo`
  - `prices.service` - Name of the sevice
  - `prices.tier` - Tier of the service. A service can be offered in several tiers. Higher tiers give customers more features.
  - `prices.promo` - Promotion which can be applied to a service/tier combination to offer a discount to customers.
  - `prices.price` - The price of a particular service/tier/promo combination.
- `services` - Services which each customer is subscribed. Primary Key: `cust_id, service`; Foreign Keys: `cust_id` references `customers.id`, (`service, tier`) references (`prices.service, prices.tier`)
  - `services.cust_id` - id of the customer associated with this subscription.
  - `services.service` - name of service associated with this subscription.
  - `services.tier` - tier of service associated with a subscription.
- `active_promos` - the promotion which is actually applied to each customer for a particular service
  - `'cust_id'` - identifies an individual customer.
  - `'service'` - identifies a service for which the customer has an active promotion. **The customer may not actually be subscribed to the service!**
  - `'promo'` - the active promo for the `cust_id`/`service` pair.
    - If `'time_left'` is '0' for all records associated with the `cust_id`/`service` pair in `promos`, this column should have a value of `'base'`.
    - If there is a record associated with a non-zero `'time_left'`, this column should have the `'promo'` from that record.

## Exercise 1 - (**2** Points):  

**Requirements**:  
Define the function `denormalize(customers, services, active_promos, prices)` which takes the DataFrame inputs defined above. See the data model diagram for the relationships between the 4 tables.

![data_model](https://github.com/gt-cse-6040/bootcamp/blob/main/Module%201/Session%209/resources/schema.png?raw=1)

  Your function should return a DataFrame `df` which contains the following columns:
  - `id` - identifies a particular customer (from `customers`)
  - `paid` - ('True'|'False') indicating whether the customer `id` has paid their bill (from `customers`)
  - `service` - a service which a customer is subscribed. There should be one record for each unique `id`/`service` pair (from `services`)
  - `tier` - tier of a service for the `id`/`service` pair. (from `services`)
  - `promo` - promo being applied to the `id`/`service` pair. (from `active_promos`)
      - Remember that a record existing with a `cust_id`/`service` combination in `active_promos` _does not imply the customer is subscribed to that service_.
  - `price` - price charged for the `id`/`service` pair (from `prices`)

In [4]:
### Define demo inputs

demo_customers_ex1 = pd.DataFrame({'id': {0: '0', 1: '1'}, 'paid': {0: 'True', 1: 'False'}})
demo_active_promos_ex1 = pd.DataFrame({'cust_id': {0: '0', 1: '0', 2: '1', 3: '1'},
 'service': {0: 'audio', 1: 'video', 2: 'audio', 3: 'video'},
 'promo': {0: 'intro', 1: 'base', 2: 'base', 3: 'intro'}})
demo_prices_ex1 = pd.DataFrame(
    {'service': {0: 'audio',  1: 'audio',  2: 'audio',  3: 'audio',  4: 'video',  5: 'video',  6: 'video',  7: 'video'},
    'tier': {0: '1', 1: '1', 2: '2', 3: '2', 4: '1', 5: '1', 6: '2', 7: '2'},
    'promo': {0: 'base', 1: 'intro', 2: 'base', 3: 'intro', 4: 'base', 5: 'intro', 6: 'base', 7: 'intro'},
    'price': {0: '8.99', 1: '5.99', 2: '12.99', 3: '9.99', 4: '10.99', 5: '8.99', 6: '15.99', 7: '11.99'}})
demo_services_ex1 = pd.DataFrame({'cust_id': {0: '0', 1: '1', 2: '1'},
 'service': {0: 'audio', 1: 'video', 2: 'audio'},
 'tier': {0: '1', 1: '1', 2: '2'}})

print('customers')
print(demo_customers_ex1)
print()
print('services')
print(demo_services_ex1)
print()
print('active_promos')
print(demo_active_promos_ex1)
print()
print('prices')
print(demo_prices_ex1)

customers
  id   paid
0  0   True
1  1  False

services
  cust_id service tier
0       0   audio    1
1       1   video    1
2       1   audio    2

active_promos
  cust_id service  promo
0       0   audio  intro
1       0   video   base
2       1   audio   base
3       1   video  intro

prices
  service tier  promo  price
0   audio    1   base   8.99
1   audio    1  intro   5.99
2   audio    2   base  12.99
3   audio    2  intro   9.99
4   video    1   base  10.99
5   video    1  intro   8.99
6   video    2   base  15.99
7   video    2  intro  11.99


In [5]:
outline = '''
output:
    - df
        - data_type == DataFrame
        - cols:
            - customers.id, customers.paid
            - services.service, services.tier
            - active_promos.promo of a subscribed id/service pair
            - prices.price of an id/service pair
        - grain == one row per cust_id + service
        - column types are strings
inputs:
    - customers, services, active_promos, prices
        - data_type == DataFrame
steps:
    1. we want to return all customers + the services they're currently paying for
        - JOIN services
    2. we want the active promo for the current services
        - JOIN active_promos
    3. we want the prices for the active promo
        - JOIN prices
'''

In [6]:
### Exercise 1 solution
def denormalize(customers, services, active_promos, prices):
    ###
    ### YOUR CODE HERE
    ###

    conn = dfs_to_conn({'customers': customers, 'services': services, 'active_promos': active_promos, 'prices': prices})
    query = '''
    SELECT
        c.id
      , c.paid
      , s.service
      , s.tier
      , ap.promo
      , p.price
    FROM customers c
    JOIN services s
      ON c.id = s.cust_id
    JOIN active_promos ap
      ON c.id = ap.cust_id
    JOIN prices p
      ON s.service = p.service
    '''

    df = pd.read_sql_query(query, conn)
    return df

demo_ex1_output = denormalize(demo_customers_ex1, demo_services_ex1, demo_active_promos_ex1, demo_prices_ex1)
print(demo_ex1_output)

   id   paid service tier  promo  price
0   0   True   audio    1   base  12.99
1   0   True   audio    1   base   5.99
2   0   True   audio    1   base   8.99
3   0   True   audio    1   base   9.99
4   0   True   audio    1  intro  12.99
5   0   True   audio    1  intro   5.99
6   0   True   audio    1  intro   8.99
7   0   True   audio    1  intro   9.99
8   1  False   audio    2   base  12.99
9   1  False   audio    2   base   5.99
10  1  False   audio    2   base   8.99
11  1  False   audio    2   base   9.99
12  1  False   audio    2  intro  12.99
13  1  False   audio    2  intro   5.99
14  1  False   audio    2  intro   8.99
15  1  False   audio    2  intro   9.99
16  1  False   video    1   base  10.99
17  1  False   video    1   base  11.99
18  1  False   video    1   base  15.99
19  1  False   video    1   base   8.99
20  1  False   video    1  intro  10.99
21  1  False   video    1  intro  11.99
22  1  False   video    1  intro  15.99
23  1  False   video    1  intro   8.99


<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 1. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [13]:
### test_cell_ex1

exercise_start = time.time()
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_1',
    'func': denormalize, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'customers':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        },
        'services':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        },
        'active_promos':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        },
        'prices':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'6IRWMcPsVIAZqzDJnPgv_MfUZsxqo4Utjm2Favidv-A=', path='resources/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

exercise_end = time.time()
print(f"This test executed in {(pd.to_datetime(exercise_end, unit='s') - pd.to_datetime(exercise_start, unit='s')).seconds} seconds")
print('Passed! Please submit.')

AssertionError: ignored

## Exercise 3 - (**2** Points):  
**Motivation** (don't dwell on this):  

The first task in our journaling process is to identify which records in the existing journal are active and which records are not. We will do so by checking the `'exp_dt'` column. All records with `'9999-12-31'` as their expiration date are considered active. We will be rebuilding the entire journal, so we need to partition the existing journal into active and not-active records and return both parts. The active records will be compared with the business data, and the inactive records will be included in the updated journal without modification. Additionally, on the initial load, there will not be an existing journal, so we will need to create it based on the data being loaded and the desired audit columns.

**Requirements**:  
Define `partition_journal(df, audit_cols, existing_journal=None)`.

- The input `df` is a DataFrame - we do not care about it's structure.
- The input `audit_cols` is a `list` of strings. These are the names of audit columns used to track history in the journal. `audit_cols` will always include the strings `'eff_dt'` and `'exp_dt'`.
- The optional input `existing_journal` is a DataFrame or `None`.  If `existing_journal` is not `None` it will have all of the columns in `df` and all of the `audit_cols` as its columns.

Your function should do the following:
- If `existing_journal` is `None`, create an empty DataFrame which has all of the columns in `df` and all of the `audit_cols` as its columns. This empty DataFrame will be used in the subsequent operations.
- Create `historical_journal` which is a DataFrame containing all rows of `existing_journal` where `'exp_dt'` is something other than `'9999-12-31'`.
- Create `active_journal` which is a DataFrame containing all rows of `existing_journal` where `'exp_dt'` is `'9999-12-31'`.
- Return the tuple `(historical_journal, active_journal)` - If the `existing_journal` was newly created these will be two empty DataFrames with all columns present in `df` and all of the `audit_cols`.

In [ ]:
### Define demo inputs

demo_df_ex3 = pd.DataFrame({'id': {0: '1', 1: '2', 2: '2'},
 'paid': {0: 'True', 1: 'True', 2: 'True'},
 'service': {0: 'audio', 1: 'video', 2: 'audio'},
 'tier': {0: '1', 1: '2', 2: '2'},
 'promo': {0: 'base', 1: 'base', 2: 'base'},
 'price': {0: '8.99', 1: '15.99', 2: '12.99'}})
demo_existing_journal_ex3 = pd.DataFrame(
    {'id': {667: '0', 668: '1', 669: '2', 670: '2', 671: '3', 672: '3', 673: '4', 9: '3', 10: '3', 17: '0', 1881: '1', 1882: '2', 1883: '2', 1884: '4'},
    'paid': {667: 'True', 668: 'True', 669: 'True', 670: 'True', 671: 'True', 672: 'True', 673: 'True', 9: 'False', 10: 'False', 17: 'True',
            1881: 'True', 1882: 'True', 1883: 'True', 1884: 'True'},
    'service': {667: 'video', 668: 'audio', 669: 'video', 670: 'audio', 671: 'video', 672: 'audio', 673: 'audio', 9: 'video', 10: 'audio', 17: 'video',
            1881: 'audio', 1882: 'video', 1883: 'audio', 1884: 'audio'},
    'tier': {667: '2', 668: '1', 669: '2', 670: '2', 671: '1', 672: '1', 673: '1', 9: '1', 10: '1', 17: '2', 1881: '1', 1882: '2', 1883: '2', 1884: '1'},
    'promo': {667: 'intro', 668: 'intro', 669: 'intro', 670: 'intro', 671: 'intro', 672: 'intro', 673: 'intro', 9: 'base', 10: 'base', 17: 'base',
            1881: 'base', 1882: 'base', 1883: 'base', 1884: 'base'},
    'price': {667: '11.99', 668: '5.99', 669: '11.99', 670: '9.99', 671: '8.99', 672: '5.99', 673: '5.99', 9: '10.99', 10: '8.99', 17: '15.99',
            1881: '8.99', 1882: '15.99', 1883: '12.99', 1884: '8.99'},
    'eff_dt': {667: '2018-02-01', 668: '2018-02-01', 669: '2018-02-01', 670: '2018-02-01', 671: '2018-02-01', 672: '2018-02-01', 673: '2018-02-01',
            9: '2018-08-01', 10: '2018-08-01', 17: '2018-08-01', 1881: '2018-08-01', 1882: '2018-08-01', 1883: '2018-08-01', 1884: '2018-08-01'},
    'exp_dt': {667: '2018-07-31', 668: '2018-07-31', 669: '2018-07-31', 670: '2018-07-31', 671: '2018-07-31', 672: '2018-07-31', 673: '2018-07-31',
            9: '2018-08-31', 10: '2018-08-31', 17: '2019-02-28', 1881: '9999-12-31', 1882: '9999-12-31', 1883: '9999-12-31', 1884: '9999-12-31'}})
demo_audit_cols_ex3 = ['eff_dt', 'exp_dt']

print('df')
print(demo_df_ex3)
print()
print('audit_cols')
print(demo_audit_cols_ex3)
print()
print('existing_journal')
print(demo_existing_journal_ex3)

<!-- Expected demo output text block -->
The demo included in the solution cell below should display the following output:
```
historical_journal WITH NO existing journal
Empty DataFrame
Columns: [id, paid, service, tier, promo, price, eff_dt, exp_dt]
Index: []

active_journal WITH NO existing journal
Empty DataFrame
Columns: [id, paid, service, tier, promo, price, eff_dt, exp_dt]
Index: []

historical_journal WITH existing journal
    id   paid service tier  promo  price      eff_dt      exp_dt
667  0   True   video    2  intro  11.99  2018-02-01  2018-07-31
668  1   True   audio    1  intro   5.99  2018-02-01  2018-07-31
669  2   True   video    2  intro  11.99  2018-02-01  2018-07-31
670  2   True   audio    2  intro   9.99  2018-02-01  2018-07-31
671  3   True   video    1  intro   8.99  2018-02-01  2018-07-31
672  3   True   audio    1  intro   5.99  2018-02-01  2018-07-31
673  4   True   audio    1  intro   5.99  2018-02-01  2018-07-31
9    3  False   video    1   base  10.99  2018-08-01  2018-08-31
10   3  False   audio    1   base   8.99  2018-08-01  2018-08-31
17   0   True   video    2   base  15.99  2018-08-01  2019-02-28

active_journal WITH existing journal
     id  paid service tier promo  price      eff_dt      exp_dt
1881  1  True   audio    1  base   8.99  2018-08-01  9999-12-31
1882  2  True   video    2  base  15.99  2018-08-01  9999-12-31
1883  2  True   audio    2  base  12.99  2018-08-01  9999-12-31
1884  4  True   audio    1  base   8.99  2018-08-01  9999-12-31
```
**Note** - This demo runs your solution two times. The first two DataFrames are the expected result when `exixting_journal` is `None`, and the second two DataFrames are the expected result for the `existing_journal` defined in the cell above.

In [ ]:
outline = '''
output:
    - (historical_journal, active_journal)
        - data_type == tuple of dataframes
inputs:
    - df
        - data_type == DataFrame
    - audit_cols
        - data_type == list of strings
        - will always include eff_dt and exp_dt
    - existing_journal
        - data_type == df or None
        - optional
steps:
    1. check if existing_journal is None
        a. if yes: create empty df where existing_journal cols = df.cols + audit_cols
    2. create journals
        a. historical_journal = existing_journal where exp_dt != '9999-12-31'
        b. active_journal = existing_journal where exp_dt = '9999-12-31'
    3. return dfs
'''

In [ ]:
### Exercise 3 solution
def partition_journal(df, audit_cols, existing_journal=None):
    ###
    ### YOUR CODE HERE
    ###

    if existing_journal is None:
        list = list(df.columns)
        empty_journal_cols = list + audit_cols
        existing_journal = pd.DataFrame(columns=empty_journal_cols)

    historical_journal = existing_journal[existing_journal['exp_dt'] != '9999-12-31']
    active_journal = existing_journal[existing_journal['exp_dt'] = '9999-12-31']

    return (active_journal, historical_journal)

### demo function call
new_hist, new_active = partition_journal(demo_df_ex3, demo_audit_cols_ex3)
hist, active = partition_journal(demo_df_ex3, demo_audit_cols_ex3, demo_existing_journal_ex3)
print('historical_journal WITH NO existing journal')
print(new_hist)
print()
print('active_journal WITH NO existing journal')
print(new_active)
print()
print('historical_journal WITH existing journal')
print(hist)
print()
print('active_journal WITH existing journal')
print(active)

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 3. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex3
exercise_start = time.time()
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_3',
    'func': partition_journal, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'df':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        },
        'audit_cols':{
            'dtype':'list', # data type of param.
            'check_modified':True,
        },
        'existing_journal':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'historical_journal':{
            'index':0,
            'dtype':'pd.DataFrame',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        },
        'active_journal':{
            'index':1,
            'dtype':'pd.DataFrame',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }

    }
}
tester = Tester(conf, key=b'6IRWMcPsVIAZqzDJnPgv_MfUZsxqo4Utjm2Favidv-A=', path='resources/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

### END HIDDEN TESTS
exercise_end = time.time()
print(f"This test executed in {(pd.to_datetime(exercise_end, unit='s') - pd.to_datetime(exercise_start, unit='s')).seconds} seconds")
print('Passed! Please submit.')

## Exercise 5 - (**3** Points):
**Motivation** (don't dwell on this):  
Our next task is to identify records which have changed and which records are unchanged. These are a subset of records having keys in both the business data and the active journal. We need to partition both the business data and journal data into two parts based on whether the data has changed.

**Requirements**:  
Define `compare_changes(compare_new_df, compare_old_df, audit_cols)`.

The inputs are as follows:

- `compare_new_df` - a DataFrame
- `compare_old_df` - another DataFrame with the same columns/shape/indexing as `compare_new_df`
- `audit_cols` - a list of column names which should not be used for comparison.

You can assume that the rows `compare_new_df` and `compare_old_df` are sorted and indexed such that they can be compared directly.

- Identify the columns in `compare_new_df` which are not in `audit_cols`. Let's call this `cols`.
- Compare the values in `compare_new_df[cols]` with the values in `compare_old_df[cols]`.
- Return these 3 new DataFrames:
    - `unchanged` - All of the rows in `compare_new_df`  where **all** values are the same in the comparison.  
    - `old_changed` - All of the rows in `compare_old_df` where there are **any** differences in the comparison.
    - `new_changed` - All of the rows in `compare_new_df` where there are **any** differences in the comparison.

It is possible that `compare_new_df` and `compare_old_df` are **both** empty DataFrames. If this is the case all 3 returned DataFrames would also be empty.



In [ ]:
### Define demo inputs

demo_compare_new_df_ex5 = pd.DataFrame([
    {'some_column': 'new_val_0', 'key_column': 'changed_0', 'audit_column_1': None, 'audit_column_2': None},
    {'some_column': 'new_val_1', 'key_column': 'changed_1', 'audit_column_1': None, 'audit_column_2': None},
    {'some_column': 'same_val_0', 'key_column': 'unchanged_0', 'audit_column_1': None, 'audit_column_2': None},
    {'some_column': 'same_val_1', 'key_column': 'unchanged_1', 'audit_column_1': None, 'audit_column_2': None},
    {'some_column': 'new_val_2', 'key_column': 'changed_2', 'audit_column_1': None, 'audit_column_2': None},
    {'some_column': 'same_val_2', 'key_column': 'unchanged_2', 'audit_column_1': None, 'audit_column_2': None}
])

demo_compare_old_df_ex5 = pd.DataFrame([
    {'some_column': 'old_val_0', 'key_column': 'changed_0', 'audit_column_1': 'foo', 'audit_column_2': 'bar'},
    {'some_column': 'old_val_1', 'key_column': 'changed_1', 'audit_column_1': 'foo', 'audit_column_2': 'bar'},
    {'some_column': 'same_val_0', 'key_column': 'unchanged_0', 'audit_column_1': 'foo', 'audit_column_2': 'bar'},
    {'some_column': 'same_val_1', 'key_column': 'unchanged_1', 'audit_column_1': 'foo', 'audit_column_2': 'bar'},
    {'some_column': 'old_val_2', 'key_column': 'changed_2', 'audit_column_1': 'foo', 'audit_column_2': 'bar'},
    {'some_column': 'same_val_2', 'key_column': 'unchanged_2', 'audit_column_1': 'foo', 'audit_column_2': 'bar'}
])

demo_audit_cols_ex5 = ['audit_column_1', 'audit_column_2']

print('compare_new_df')
print(demo_compare_new_df_ex5)
print()
print('compare_old_df')
print(demo_compare_old_df_ex5)
print()
print('audit_cols')
print(demo_audit_cols_ex5)

In [ ]:
outline = '''
output:
    - (unchanged, old_changed, new_changed)
        - data_type == DataFrames
inputs:
    - compare_new_df
        - data_type == DataFrame
    - compare_old_df
        - data_type == DataFrame
    - audit_cols
        - data_type == list
steps:
    1. check to see if input dfs are empty
        a. if empty, return 3 empty DataFrames with the right columns names
    2. find cols that are not in audit_cols
        a. get list of columns from compare_new_cols
        b. if col in compare_new_cols not in audit_cols:
            - add to new list, cols
    3. create boolean mask - True when there is any difference between the two dfs[cols]
    4. use different to partition the dfs
    5. return (unchanged, old_changed, new_changed)
'''

In [ ]:
### Exercise 5 solution
def compare_changes(compare_new_df, compare_old_df, audit_cols):
    ###
    ### YOUR CODE HERE
    ###

    # check for empty inputs
    empty_df = pd.DataFrame()
    if compare_new_df.shape[0] == compare_old_df.shape[0] == 0:
        return (empty_df, empty_df, empty_df)

    # find cols that are not in audit_cols
    compare_new_cols = list(compare_new_df.columns)
    cols = []
    for i in compare_new_cols:
        if i not in audit_cols:
            cols.append(i)

    # create boolean mask - True when there is any difference between the two frames, ignoring audit_cols
    different = (compare_new_df[cols] != compare_old_df[cols])

    # use different to partition the dfs
    unchanged = compare_new_df.iloc[~different, :]
    old_changed = compare_old_df.iloc[different, :]
    new_changed = compare_new_df.iloc[different, :]

    # return
    return (unchanged, old_changed, new_changed)

# Run demo of function
(demo_unchanged_ex5,
demo_old_changed_ex5,
demo_new_changed_ex5) = compare_changes(demo_compare_new_df_ex5, demo_compare_old_df_ex5, demo_audit_cols_ex5)
print('unchanged')
print(demo_unchanged_ex5)
print()
print('old_changed')
print(demo_old_changed_ex5)
print()
print('new_changed')
print(demo_new_changed_ex5)

In [ ]:
### test_cell_ex5
exercise_start = time.time()
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_5',
    'func': compare_changes, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'compare_new_df':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        },
        'compare_old_df':{
            'dtype':'pd.DataFrame', # data type of param.
            'check_modified':True,
        },
        'audit_cols':{
            'dtype':'list', # data type of param.
            'check_modified':True,
        }
    },
    'outputs':{
        'unchanged':{
            'index':0,
            'dtype':'pd.DataFrame',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        },
        'old_changed':{
            'index':1,
            'dtype':'pd.DataFrame',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        },
        'new_changed':{
            'index':2,
            'dtype':'pd.DataFrame',
            'check_dtype': True,
            'check_col_dtypes': False, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'6IRWMcPsVIAZqzDJnPgv_MfUZsxqo4Utjm2Favidv-A=', path='resources/')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

exercise_end = time.time()
print(f"This test executed in {(pd.to_datetime(exercise_end, unit='s') - pd.to_datetime(exercise_start, unit='s')).seconds} seconds")
print('Passed! Please submit.')

**Fin!** You’ve reached the end of this part. Don’t forget to restart and run all cells again to make sure it’s all working when run in sequence; and make sure your work passes the submission process. Good luck!